In [1]:
###modify before class to include correct path
#https://drive.google.com/file/d/13A41P-RsqmNTawrnMlCBVrQAX7XW-HGl/view?usp=sharing
zip_id = "13A41P-RsqmNTawrnMlCBVrQAX7XW-HGl"
!gdown --id {zip_id} -O reduced_dataset.zip
# חילוץ הקבצים
!unzip reduced_dataset.zip -d reduced_dataset

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=13A41P-RsqmNTawrnMlCBVrQAX7XW-HGl
From (redirected): https://drive.google.com/uc?id=13A41P-RsqmNTawrnMlCBVrQAX7XW-HGl&confirm=t&uuid=c211b2ef-f57f-41b3-b953-e02bd1a4e46b
To: /content/reduced_dataset.zip
100% 40.9M/40.9M [00:00<00:00, 67.8MB/s]
Archive:  reduced_dataset.zip
   creating: reduced_dataset/beef_tartare/
   creating: reduced_dataset/churros/
   creating: reduced_dataset/french_onion_soup/
   creating: reduced_dataset/greek_salad/
   creating: reduced_dataset/guacamole/
   creating: reduced_dataset/pad_thai/
   creating: reduced_dataset/pancakes/
   creating: reduced_dataset/pho/
   creating: reduced_dataset/strawberry_shortcake/
   creating: reduced_dataset/sushi/
  inflating: reduced_dataset/str

In [ ]:
zip_id = "1-329XUOi4XaagdlutAqvPi4yKB-bX9U9"
!gdown --id {zip_id} -O dataset.zip
#https://drive.google.com/file/d/1-329XUOi4XaagdlutAqvPi4yKB-bX9U9/view?usp=drive_link
# חילוץ הקבצים
!unzip dataset.zip -d dataset
print('-'*20+'\n Upload Completed!!\n'+'-'*20)

In [4]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from tensorflow.keras import Sequential

In [4]:
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.18.0
Keras version: 3.8.0


In [3]:
!pip uninstall -y tensorflow keras tensorflow-hub
!pip install tensorflow==2.15 keras==2.15 tensorflow-hub
#ואז לעשות restart session

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: tensorflow-hub 0.16.1
Uninstalling tensorflow-hub-0.16.1:
  Successfully uninstalled tensorflow-hub-0.16.1
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [5]:
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.15.0
Keras version: 2.15.0


In [6]:
#adjust pths if needed
reduced_dataset_path = 'reduced_dataset'
test_path = 'dataset/test'
IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

# apply data augmentation
# train datagen
train_datagen = ImageDataGenerator(
    rescale=1./255
)

# test datagen
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# train generator
train_generator = train_datagen.flow_from_directory(
    reduced_dataset_path,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=42
)

# test generator
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=42
)

Found 800 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [7]:
model_url = 'https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1'
model2_url = "https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1"

model_url = 'some_url...'
feature_extractor_layer = hub.KerasLayer(model_url, trainable=False, input_shape=(224, 224, 3))
num_classes =10

resnet_model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [8]:
# compile resenet_model
resnet_model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# train resenet_model
history = resnet_model.fit(
    train_generator,
    epochs=10, #do 3 it is slow af
    validation_data=test_generator,
    verbose=1#,
    #callbacks=[early_stopping]
)

# summary
resnet_model.summary()


Epoch 1/10
25/25 [==============================] - 437s 18s/step - loss: 1.6362 - accuracy: 0.4712 - val_loss: 0.7997 - val_accuracy: 0.7650
Epoch 2/10
25/25 [==============================] - 439s 18s/step - loss: 0.5565 - accuracy: 0.8438 - val_loss: 0.5288 - val_accuracy: 0.8400
Epoch 3/10
25/25 [==============================] - 434s 18s/step - loss: 0.3546 - accuracy: 0.9237 - val_loss: 0.4557 - val_accuracy: 0.8605
Epoch 4/10
25/25 [==============================] - 427s 18s/step - loss: 0.2636 - accuracy: 0.9388 - val_loss: 0.4232 - val_accuracy: 0.8710
Epoch 5/10
25/25 [==============================] - 444s 18s/step - loss: 0.1895 - accuracy: 0.9688 - val_loss: 0.3980 - val_accuracy: 0.8800
Epoch 6/10
25/25 [==============================] - 446s 18s/step - loss: 0.1509 - accuracy: 0.9812 - val_loss: 0.3941 - val_accuracy: 0.8815
Epoch 7/10
25/25 [==============================] - 426s 18s/step - loss: 0.1246 - accuracy: 0.9887 - val_loss: 0.3788 - val_accuracy: 0.8770
Epoch 